In [1]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

In [15]:
import glob
import itertools
import sys
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

sys.path.append('..')

from run import Run
from run_db import db as run_db
from run_utils import init_runs, find_runs

tasks = ['dep', 'lmo', 'ner', 'pos']
langs = ['cs', 'de', 'en', 'es']

log_path = '/home/mpikuliak/logs/'

runs = init_runs(log_path, run_db)

np.set_printoptions(precision=2)

In [3]:
def draw_graphs(runs, tasks, langs, role, metric=None, focused=False, label=None):
    
    if label is None:
        label = lambda run: f'{run.name}-{run.type}'
    
    fig, axes = plt.subplots(
        len(tasks),
        len(langs),
        figsize=(5*len(langs), 4*len(tasks)),
        squeeze=False)

    for ax_row, task in zip(axes, tasks):
        for ax, lang in zip(ax_row, langs):
            for run in runs:
                if not focused or run.config['focus_on'] == f'{task}-{lang}':
                    history = run.history(
                        metric=metric,
                        task=task,
                        language=lang,
                        role=role)
                    ax.plot(list(history), label=label(run))

    for ax, lang in zip(axes[0], langs):
        ax.set_title(lang)

    for ax, task in zip(axes[:, 0], tasks):
        ax.set_ylabel(task, rotation=0, size='large')
        
    for ax_row in axes:
        for ax in ax_row:
            ax.legend()

    plt.show()

    
def results(runs, tasks, langs, focused=True, values_only=False):
    
    def get_results(run):
        out = {}
        for task, lang in itertools.product(tasks, langs):
            run.load()
            if (not focused and run.contains(task, lang)) or run.config['focus_on'] == f'{task}-{lang}':
                res, epoch = run.metric_eval(task=task, language=lang)
                if res <= 1.01:
                    res *= 100
                out[(task, lang)] = res, epoch
        return out
    
    out = {
        run: get_results(run)
        for run in runs}
    
    if values_only:
        return np.array([
            list(run.values())[0][0] for run in out.values()
        ])
    else:
        return out
    
def np_results(name, type='all', focused=True):
    return results(
        runs=find_runs(
            runs,
            type=type,
            name=name
        ),
        tasks=tasks,
        langs=langs,
        values_only=True,
        focused=focused
    )

def gen_table(runs, upper, baseline=None):
    metrics = np.vstack([
        np.array(np_results(code, type)) for _, type, code in runs
    ])
    
    if baseline is None:
        baseline = metrics[0]
    aer = (metrics - baseline) / (upper - baseline)
    
    return np.vstack([
        np.mean(metrics[:,:4], axis=1),
        np.mean(metrics[:,4:8], axis=1),
        np.mean(metrics[:,8:], axis=1),
        np.mean(aer, axis=1)*100
    ]).T


def tempo(name, type='all', mean=True):    
    out = np_results(name, type)
    for value in out:
        print(f'{value:.2f} ', end='')
    if mean:
        print(np.round([np.mean(out[:4]), np.mean(out[4:8]), np.mean(out[8:])], decimals=2), end='')
    print()
    
simple_baseline = np_results('normal-training', 'stsl', focused=False)
two_two_baseline = simple_baseline[[11, 9, 11, 0, 6, 3, 0, 6, 7, 8, 7, 8, 8, 3, 2, 2, 6, 10, 9, 0,]]
print(simple_baseline)

[86.52 76.42 82.44 85.48 69.06 78.52 81.48 79.7  97.6  92.61 94.36 96.23]


In [4]:
two_by_two = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

adv = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two-adversarial'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

for i, val in enumerate(two_by_two):
    print(f'{val:.2f}', end=' ')
    if i % 6 == 5:
        print(f'{adv[i // 6]:.2f}')

49.37 57.57 48.39 81.51 84.35 86.67 73.18
60.08 66.73 61.53 67.83 68.99 64.72 74.82
50.60 52.56 47.21 54.21 56.60 43.19 66.19
22.65 23.97 22.60 24.86 27.45 25.11 37.19
46.28 47.66 47.70 47.37 46.99 47.68 48.10
20.13 16.41 20.04 26.26 35.77 29.42 47.25
22.70 22.05 21.74 23.67 33.09 21.93 40.75
44.87 41.23 42.86 45.09 45.47 49.44 47.23
38.92 42.72 39.72 48.34 48.79 30.72 58.35
62.04 67.09 62.05 64.51 67.51 65.12 71.72
30.36 32.69 30.28 47.93 46.32 22.29 51.61
56.61 64.42 62.70 66.67 66.54 63.83 71.50
58.53 63.15 59.64 78.45 80.83 81.18 80.64
22.37 20.42 22.53 24.13 31.67 16.91 53.89
31.31 35.42 30.60 37.00 52.58 49.25 52.51
31.08 27.65 31.58 27.37 37.67 33.11 42.51
44.00 43.05 42.02 45.82 46.98 43.64 44.64
62.49 69.67 59.84 69.72 69.68 55.69 73.61
70.70 77.97 72.97 82.37 82.51 87.24 86.27
24.46 23.35 23.05 33.84 45.11 45.67 51.33


In [ ]:
tempo('zero-shot-embs', 'rel')
tempo('zero-shot-embs-again', 'rel')



In [ ]:
tmp = find_runs(runs, name='zero-shot')
print(tmp[0].data[0])
draw_graphs(tmp, tasks, langs, 'test', metric='loss', focused=True)


In [ ]:
tmp = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmp2 = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource-2'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmpa = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource-advanced'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

adv = iter(tmpa)
baselines = iter(tmp[0::5] + tmp2[0::5])
bsl = next(baselines)

i = 0
for r in tmp:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 5 == 0:
        for _ in range(3):
           print(f'{next(adv)/bsl:.2f}', end=' ')     
        print()
        bsl = next(baselines)
for r in tmp2:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 5 == 0:
        for _ in range(3):
           print(f'{next(adv)/bsl:.2f}', end=' ')    
        print()
        bsl = next(baselines)



In [ ]:
tmp = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmpa = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two-adversarial'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

baselines = iter(tmp[0::6])
bsl = next(baselines)
adv = iter(tmpa)

i = 0
for r in tmp:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 6 == 0:
        print(f'{next(adv)/bsl:.2f}')   
        bsl = next(baselines)




In [ ]:
# Porovnanie CL, CL+CT, CL+Co, CL+CT+Co trenovania pre 4x4 a 2x2 nastavenie.
print(gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'rel', 'zero-shot'),
    (12, 'ml-unrel-12', 'zero-shot'),
    (12, 'all', 'zero-shot'),
    (12, 'all', 'zero-shot-400'),  
], simple_baseline))
                       


mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,:4].T
# print(mat)
mat = (mat - mat[0]) / (two_two_baseline - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)

In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot-task'),
    (12, 'all', 'zero-shot-lang'),
    (12, 'all', 'zero-shot-task-lang-no-global'), 
    (12, 'all', 'zero-shot-task-lang'),
], simple_baseline)

print(tabl)

mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,[0,-1]].T
mat = (mat - mat[0]) / (two_two_baseline - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)
                      

In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot-task-emb'),
    (12, 'all', 'zero-shot-lang-emb'),
    (12, 'all', 'zero-shot-embs'),
], simple_baseline)

print(tabl)

mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,[0,-2]].T
mat = (mat - mat[0]) / (two_two_baseline - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)


In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot-adversarial'),
], simple_baseline)

print(tabl)

mat = np.vstack([
    np.reshape(
        np_results('zero-shot-two-by-two', 'var'), (20, 6)
    )[:,0],
    np_results('zero-shot-two-by-two-adversarial', 'var')
])
mat = (mat - mat[0]) / (two_two_baseline - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)


In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot-task-lang'),
    (12, 'all', 'zero-shot-embs-400'),
    ...
], simple_baseline)

print(tabl)

In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot'),
    (12, 'ml-3', 'zero-shot-rotated'),    
    (12, 'all', 'zero-shot-rotated'),
    (12, 'all', 'zero-shot-task-lang-rotated'),
    (12, 'all', 'zero-shot-embs-rotated'),
    (12, 'all', 'zero-shot-rotated-adversarial'),
    (12, 'ml-3', 'zero-shot-char-level'),
    (12, 'all', 'zero-shot-char-level'),
    (12, 'all', 'zero-shot-task-lang-char-level'),        
    (12, 'all', 'zero-shot-embs-char-level'),
    (12, 'all', 'zero-shot-char-level-adversarial'),
], simple_baseline)

for row in tabl:
    for num in row:
        print(' & ', f'{num:.2f}', end='')
    print(' \\\\')

In [ ]:
tabl = gen_table([
    (12, 'ml-3', 'zero-shot'),
    (12, 'all', 'zero-shot'),
    (12, 'all', 'zero-shot-limited-task-200'),
    (12, 'all', 'zero-shot-task-lang-limited-task-200'),
    (12, 'all', 'zero-shot-embs-limited-task-200'),
    (12, 'all', 'zero-shot-adversarial-limited-task-200'),
    (12, 'all', 'zero-shot-limited-lang-200'),
    (12, 'all', 'zero-shot-task-lang-limited-lang-200'),
    (12, 'all', 'zero-shot-embs-limited-lang-200'),
    (12, 'all', 'zero-shot-adversarial-limited-lang-200'),
], simple_baseline)

for row in tabl:
    for num in row:
        print(' & ', f'{num:.2f}', end='')
    print(' \\\\')

In [21]:
tabl = gen_table([
        (12, 'ml-3', 'low-resource'),
        (12, 'mt-3', 'low-resource'),
        (12, 'rel', 'low-resource'),
        (12, 'all', 'low-resource'),
        (12, 'all', 'low-resource-task-lang'),
        (12, 'all', 'low-resource-embs'),
        (12, 'all', 'low-resource-adversarial'),
], simple_baseline)

for row in tabl:
    for num in row:
        print(' & ', f'{num:.2f}', end='')
    print(' \\\\')


[86.52 76.42 82.44 85.48 69.06 78.52 81.48 79.7  97.6  92.61 94.36 96.23]
 &  64.30 &  60.16 &  90.10 &  0.00 \\
 &  65.56 &  61.91 &  91.64 &  16.16 \\
 &  70.78 &  64.82 &  92.23 &  35.36 \\
 &  71.05 &  63.02 &  92.51 &  35.43 \\
 &  68.58 &  63.21 &  92.21 &  28.02 \\
 &  70.60 &  64.06 &  92.23 &  34.23 \\
 &  69.40 &  62.02 &  91.98 &  25.97 \\
